# Assignment 2 Group no. 6
### Project members: 
Mar Balibrea Rull, marbr@kth.se
Marcos Fernández Carbonell, marcosfc@kth.se
Tiger Huan Hu Zha, tigerz2@illinois.edu

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.


### Instructions
All assignments starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of assignments starting with 
number 2 below, then the assignment will receive 2 points (in total).

It is highly recommended that you do not develop the code directly within the notebook
but that you copy the comments and test cases to your regular development environment
and only when everything works as expected, that you paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above,
and thereby 


## Load NumPy, pandas and time

In [248]:
import numpy as np
import pandas as pd
import time

## Reused functions from Assignment 1

In [249]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
def create_normalization(df, normalizationtype = "minmax"):
    
    # hints 1, 2
    _df = df.select_dtypes(include = ["float", "int"]).drop(columns = ["CLASS", "ID"])
    
    # hint 3
    if normalizationtype is "minmax":
        
        normalization = {col: (normalizationtype, _df[col].min(), _df[col].max()) for col in _df.columns}
    elif normalizationtype is "zscore":
        
        normalization = {col: (normalizationtype, _df[col].mean(), _df[col].std()) for col in _df.columns}
    
    df_out = apply_normalization(_df, normalization)
    return df_out, normalization

def apply_normalization(df, normalization):
    
    df_out = df.copy()
    for col in normalization:
        a = normalization[col][1]
        b = normalization[col][2]
    
        # hint 3
        if normalization[col][0] is "minmax":
            
            df_out[col] = (df_out[col] - a)/(b - a) # using broadcasting
            # df_out[col] = df_out[col].apply(lambda x: (x - a)/(b - a)) # using lambda
            df_out[col].clip(0, 1, inplace = True)
        elif normalization[col][0] is "zscore":
    
            df_out[col] = df_out[col].apply(lambda x: (x - a)/b)
    return df_out

def create_imputation(df):
    
    df_tmp = df.copy()
    
    df_tmp_num = df_tmp.drop(["CLASS","ID"], axis=1, errors="ignore").select_dtypes(include=["int","float"]) # Select only numeric columns
    df_mean = pd.DataFrame(df_tmp_num.mean())
    df_tmp_num[(df_tmp_num.isna().all()).index[df_tmp_num.isna().all()]] = 0 # When all values are missing in a numeric column all values are replaced with by 0

    df_tmp_obj = df_tmp.drop(["CLASS","ID"], axis=1, errors="ignore").select_dtypes(include=["object","category"]) # Select only object and categorical columns
    df_mode = df_tmp_obj.mode().transpose()
    df_tmp_obj[(df_tmp_obj.isna().all()).index[df_tmp_obj.isna().all()]] = "" # When all values are missing in a object or categorical column all values are replaced by ""

    imputation = ((pd.concat([df_mean[0],df_mode[0]], axis=0)).fillna(0)).to_dict() # Create a mapping from column name to new value
    
    df_out = apply_imputation(df, imputation) # Apply imputation
    return df_out, imputation

def apply_imputation(df, imputation):
    
    df_out = df.copy()
    df_out.fillna(value=imputation, inplace=True) # Replace NaN values accoding with the dictionary imputation
    return df_out

def create_bins(df, nobins = 10, bintype = "equal-width"):
    
    # hints 1, 2, 3
    _df = df.select_dtypes(include = ["float", "int"]).drop(columns = ["CLASS", "ID"])
    
    binning = {}
    for col in _df.columns:
        
        if bintype == "equal-width":
            
            res, bins = pd.cut(_df[col], nobins, retbins = True, labels = False, duplicates = "drop")
        elif bintype == "equal-size":
            
            res, bins = pd.qcut(_df[col], nobins, retbins = True, labels = False, duplicates = "drop")
            
        # hint 6
        bins[0] = -np.inf
        bins[-1] = np.inf
        
        binning[col] = tuple(bins)
        
    # hints 4, 5
    df_out = apply_bins(_df, binning)
    return df_out, binning

def apply_bins(df, binning):
    
    # hint 1
    df_out = df.copy()
    
    for col in binning:
        
        # hints 2, 3, 4
        df_out[col] = pd.DataFrame(pd.cut(df_out[col], binning[col], labels = False, duplicates = "drop"))
        df_out[col].astype("category") #.cat.set_categories(np.arange(len(list(binning.values())[0])))
    
    return df_out
    
def accuracy(df, correctlabels):
    
    maximums = df.idxmax(axis = 1)
    acc = np.sum(maximums == correctlabels)
    
    return acc/df.shape[0]

def create_one_hot(df):
    
    # hints 1, 2
    _df = df.select_dtypes(include = ["object", "category"]).drop(columns = ["CLASS", "ID"], errors = "ignore")
    _df = _df.astype("category")
    
    one_hot = {col: tuple(_df[col].cat.categories) for col in _df}
    df_out = apply_one_hot(df, one_hot)
    
    return df_out, one_hot

def apply_one_hot(df, one_hot):
    
    df_out = df.copy()
    
    # hint 3
    # new_cols = [col+"-"+cat for col in df.columns for cat in df[col].unique()]
    # df_out = pd.DataFrame(columns = new_cols)
    for col in one_hot:
        
        for cat in one_hot[col]:   
            
            # hint 4
            df_out[col+"-"+cat] = (df[col] == cat).astype("float")
        
        # hint 5
        df_out.drop(columns = col, inplace = True)
    
    return df_out

def brier_score(df, correctlabels):
    
    cols = df.columns
    correct = [(l == cols).astype(float) for l in correctlabels]
    _df = pd.DataFrame(correct, columns = cols).subtract(df)**2
    
    return sum(_df.sum(axis = 1))/len(df)

def auc(df, correctlabels):
    
    cols = df.columns
    correct = [(l == cols).astype("float") for l in correctlabels]
    c_df = pd.DataFrame(correct, columns = cols) # one hot

    # hint 1
    AUC = 0
    for col in df.columns:
        
        # hint 3
        pairs = {score: [0, 0] for score in df[col]} # [positive, negative]

        for r in range(len(df[col])):
            
            # hint 2
            score = df[col][r] # probability of class col row n
            is_positive = c_df[col][r] == 1
            pairs[score] = [pairs[score][0] + is_positive, pairs[score][1] + ~is_positive]

        # hints 4, 5
        sorted_list = np.array([pairs[score] for score in sorted(pairs, reverse = True)])
        tp = sorted_list[:, 0]
        fp = sorted_list[:, 1]

        AUC_tmp = 0
        cov_tp = 0
        tot_tp = sum(tp)
        tot_fp = sum(fp)
        
        # hint 6
        for i in range(len(tp)):
            
            if fp[i] == 0:
                
                cov_tp += tp[i]
            elif tp[i] == 0:
                
                AUC_tmp += (cov_tp/tot_tp)*(fp[i]/tot_fp)
            else:
                
                AUC_tmp += (cov_tp/tot_tp)*(fp[i]/tot_fp)+(tp[i]/tot_tp)*(fp[i]/tot_fp)/2
                cov_tp += tp[i]
                
        AUC += sum(c_df[col])/len(c_df)*AUC_tmp
    return AUC

## 1. Define the class kNN

In [250]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# imputation, normalization, one_hot, labels, training_labels, training_data
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype: "minmax" (default) or "zscore"
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.normalization should be a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot should be a one-hot mapping (see Assignment 1; can be excluded if this function was not completed)
# self.training_labels should be a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels should be the categories of the previous series
# self.training_data should be the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
# normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns 
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self: the object itself
# df: a dataframe
# k: an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are estimated by the relative class frequencies in the set of class labels from the k nearest 
#              (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation, normalization and (possibly) one-hot
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies


In [251]:
class kNN:
    
    def __init__(self):
        
        self.imputation = None
        self.normalization = None
        self.labels = None
        self.training_labels = None
        self.training_data = None
        
    def fit(self, df, normalizationtype = "minmax"):
        
        self.training_data, self.imputation = create_imputation(df)
        self.training_data, self.normalization = create_normalization(self.training_data, normalizationtype)
        self.training_labels = df["CLASS"].astype("category")
        self.labels = self.training_labels.cat.categories
        self.training_data.drop(columns = ["CLASS", "ID"], errors = "ignore")
        
    def predict(self, df, k = 5):
        
        def get_nearest_neighbor_predictions(x, k): # x is a row
        
            # hint 3
            distances = [(i, np.sqrt(np.sum((x - row)**2))) for i, row in self.training_data.iterrows()] # euclidean distances
            distances.sort(key = lambda tup: tup[1]) # sort by distance
            indexes = [d[0] for d in distances[:k]] # get indexes (rows)
            classes = self.training_labels.loc[indexes] # get classes of indexes
            frequencies = [np.sum(classes == c)/classes.shape[0] for c in self.labels]
            
            return frequencies
        
        # hints 1, 2
        _df = df.drop(columns = ["CLASS", "ID"])
        _df = apply_imputation(_df, self.imputation)
        _df = apply_normalization(_df, self.normalization)
        
        predictions = pd.DataFrame(0, index = np.arange(len(df)), columns = self.labels)
        for i, row in _df.iterrows():
            
            predictions.iloc[i, :] = get_nearest_neighbor_predictions(row, k)
            
        return predictions
        
        

In [252]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

results


Training time: 0.03 s.
Testing time (k=1): 5.06 s.
Testing time (k=3): 5.00 s.
Testing time (k=5): 4.94 s.
Testing time (k=7): 4.92 s.
Testing time (k=9): 4.94 s.


,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.471028,0.833843
7,0.598131,0.471867,0.833481
9,0.616822,0.482981,0.827727


In [253]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.2f}".format(brier_score(predictions,train_labels)))


Accuracy on training set (k=1): 1.00
AUC on training set (k=1): 1.00
Brier score on training set (k=1): 0.00


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [254]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.class_priors should be a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
# to the relative frequencies of the labels
# self.feature_class_value_counts should be a mapping from a feature (column name) to another mapping, which
# given a feature value and class label provides the number of training instances with this specific combination
# self.feature_class_counts should me a mapping from the feature (column name) and class label to the number of
# training instances with this specific class label and any (non-missing) value for the feature
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: feature_class_value_counts can be a dictionary, which given a feature f returns a mapping obtained 
#         by pandas groupby and size (see lecture slides), which given a feature value v and class label c 
#         returns the number of instances, e.g., using get((c,v),0)
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
# predictions with estimated class probabilities for each row in df, where the class probabilities
# are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply discretization
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors


In [243]:
class NaiveBayes:
    
    def __init__(self):
        
        self.binning = None
        self.class_priors = None
        self.feature_class_value_counts = None
        self.feature_class_counts = None
        
    def fit(self, df, nobins, bintype = "equal-width"):
        
        # hint 1
        _df, self.binning = create_bins(df, nobins, bintype)
        self.class_priors = dict(_df["CLASS"].value_counts())
        #self.class_priors = {c: np.sum(df["CLASS"] == c)/df.shape[1] for c in df["CLASS"].unique()}
        self.feature_class_value_counts = {col: {(n[0], n[1]): len(g) 
                                                 for (n, g) in _df.groupby(["CLASS", col])} 
                                           for col in _df.columns.drop(["CLASS", "ID"])}
        self.feature_class_counts = {col: {n: len(g) for (n, g) in _df.groupby("CLASS")} for col in _df.columns.drop(["CLASS", "ID"])}
        
    def predict(self, df):
        
        # hint 1
        _df = apply_bins(df, self.binning)
        
        # hint 2
        relative_freq = self.feature_class_value_counts.copy()
        for f in self.feature_class_value_counts.keys():
            
            for class_value in self.feature_class_value_counts[f].keys():
                
                    relative_freq[f][class_value] /= self.feature_class_counts[f][class_value[0]]
        
        # hint 3
        non_normalized_estimated_class_prob = pd.DataFrame(1., index = np.arange(len(_df)), columns = self.class_priors.keys()).fillna(1.)
        for i in range(_df.shape[0]):
            
            for feat in _df.columns.drop(["CLASS", "ID"]):
                
                value = _df[f].iloc[i]
                for c in self.class_priors.keys():
                    
                    non_normalized_estimated_class_prob[c][i] *= relative_freq[f].get((c,value), 0)
            non_normalized_estimated_class_prob.iloc[i] *= pd.Series(self.class_priors)
                
        # hint 4                                                                                                          
        for i in non_normalized_estimated_class_prob.index[non_normalized_estimated_class_prob.sum(axis = 1) == 0].tolist():
            
            non_normalized_estimated_class_prob.loc[i] = pd.Series(self.class_priors)
        normalized_estimated_class_prob = non_normalized_estimated_class_prob.div(non_normalized_estimated_class_prob.sum(axis = 1), axis = 0).fillna(0)
                                                                                                            
        return normalized_estimated_class_prob
        

In [255]:
class NaiveBayes:
    def __init__(self):
        self.binning = None
        self.class_priors = None
        self.feature_class_value_counts = None
        self.feature_class_counts = None
    
    def fit(self,df,nobins=10,bintype="equal-width"):
        
        df_tmp,self.binning = create_bins(df, nobins, bintype) # Get a discretization mapping
        self.class_priors = dict(df_tmp["CLASS"].value_counts()) #  Mapping (dictionary) from the labels of the "CLASS" to the relative frequencies of the labels
        self.feature_class_value_counts = {col:{(n[0],n[1]):len(g)
                                                for (n,g) in df_tmp.groupby(["CLASS",col])}
                                                for col in df_tmp.columns.drop(["CLASS","ID"])} # Mapping from a feature (column name) to another mapping, which given a feature value and class label provides the number of training instances with this specific combination
        self.feature_class_counts = {col:{n:len(g) for (n,g) in df_tmp.groupby("CLASS")} for col in df_tmp.columns.drop(["CLASS","ID"])} # Mapping from the feature (column name) and class label to the number of training instances with this specific class label and any (non-missing) value for the feature
    
    def predict(self,df):
        # 1. Apply discretization
        df_tmp = apply_bins(df,self.binning)

        # 2. Calculate relative frequency of the observed feature value given the class (using feature_class_value_counts and feature_class_counts)
        relative_freq = self.feature_class_value_counts.copy() # We don't use feature_class_value_counts anymore so we can use a shallow copy
        for feat in self.feature_class_value_counts.keys():
            for class_value in self.feature_class_value_counts[feat].keys():
                    relative_freq[feat][class_value] /= self.feature_class_counts[feat][class_value[0]]
        
        # 3. Calculate the non-normalized estimated class probabilities by multiplying the class priors to the product of the relative frequencies
        non_normalized_estimated_class_prob = pd.DataFrame(1.,index=np.arange(len(df_tmp)), columns=self.class_priors.keys()).fillna(1.)
        for i in range(df_tmp.shape[0]):
            for feat in df_tmp.columns.drop(["CLASS","ID"]):
                value = df_tmp[feat].iloc[i]
                for c in self.class_priors.keys(): # c --> CLASS
                    non_normalized_estimated_class_prob[c][i] *= relative_freq[feat].get((c,value),0) # Calculate product of the relative frequencies
            non_normalized_estimated_class_prob.iloc[i] *= pd.Series(self.class_priors) # Multiply the class priors to the product of the relative frequencies
                
        # 4. Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
        #    this sum is zero, then set the probabilities to the class priors                                                                                                            
        for i in non_normalized_estimated_class_prob.index[non_normalized_estimated_class_prob.sum(axis=1)==0].tolist(): # In case this sum is zero, then set the probabilities to the class priors
            non_normalized_estimated_class_prob.loc[i] = pd.Series(self.class_priors)
        normalized_estimated_class_prob = non_normalized_estimated_class_prob.div(non_normalized_estimated_class_prob.sum(axis=1), axis=0).fillna(0) # Normalize the probabilities
                                                                                                            
        return normalized_estimated_class_prob

In [256]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

results

KeyError: 'CLASS'

In [40]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.85
AUC on training set: 0.97
Brier score on training set: 0.23


### Comment on assumptions, things that do not work properly, etc.